# **Remote Sensing Lab : Setup a RS pipeline to respond to an actual use case !**

Vincent Delbar - Antoine Gademer - Bastien Nguyen-Duy / 2023

# Objectifs : 

PART1: (3h) Discovery of a RS pipeline
1. Find an Area of Interest by filtering geoportal commune layer --> Export the bounding box
2. Searching relevant images in the Copernicus catalog
3. Actually downloading the images from the Sentinel Dataspace
4. Image processing on Sentinel-2 images :  True-color, False-color
5. High level processing with SentinelHub library
6. Application to the analysis of the fire in La Teste-de-Buch 

PART2: (3h) Build a RS pipeline to solve your own Use case!

Prerequisite :

- Manipulate vector data in python (and understand CRS)
- Manipulate and display 16bit images in python
- Understand the concept of Spectral Signature
- Interacte with an API in python
- Need to create a free accound on SentinelHub

# 1. Find an Area of Interest

Satelitte sensors can acquire data all over the world. The data availlable in the catalogs can be overwhelming. It is therefore primordial to know WHERE you want to look.

Let's say we want to explore the impact of the fires that have impacted the Gironde department from July to August 2022.

https://fr-m-wikipedia-org.translate.goog/wiki/Feux_de_for%C3%AAt_de_2022_en_Gironde?_x_tr_sl=fr&_x_tr_tl=en&_x_tr_hl=fr&_x_tr_pto=wapp

The fire started in the municipalities of "La Teste-de-Buch" so we will concentrate our study in this zone.

## Using the geoportal WFS server

In France, National Geographic Institute (IGN) is responsible of producing, selling and sharing all geographical data concerning the France territory.

One of it's main product is the "BD TOPO", a collection of vector layers on many subjects : roads, rivers... and municipalites ("communes" in French)

There is many way to access it. For this exercise, we propose you to use the geoportal WFS API accessible through the URL : 

```
https://wxs.ign.fr/topographie/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0&request=GetFeature&OUTPUTFORMAT=application/json&typename=BDTOPO_V3:commune
```

⚠ With 35000 municipalites in France, we counsel you to add a filtering parameter at the end : 
```
&CQL_FILTER=code_insee_du_departement=33
```
(Nota Bene : 33 is the zipcode of the Gironde department)

⚠ Side note. If you read this *after December 31th 2023*, the urls/protocols will have changed. More info : https://geoservices.ign.fr/bascule-vers-la-geoplateforme

<details>

<summary>Tips</summary>

Geopandas is able to read directly the WFS url.
We can also downloaded the json file and opened it locally.

```python
municipalities = geopandas.read_file(wfs_url)
```
</details>

<details>

<summary>Help : I have a timeout error message...</summary>

If you got a timeout error message, you can also try to click on the link/open it in your web browser and save it as a JSON file locally. Then you'll be able to read it with geopandas.
</details>

In [ ]:
%pip install geopandas folium matplotlib mapclassify

In [ ]:
import geopandas as gpd
import folium
import matplotlib
import mapclassify
import matplotlib.pyplot as plt

import geopandas as gpd
import folium
import matplotlib
import 

In [ ]:
#Your code here

wfs_url = 'https://wxs.ign.fr/topographie/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0&request=GetFeature&OUTPUTFORMAT=application/json&typename=BDTOPO_V3:commune&CQL_FILTER=code_insee_du_departement=33'
municipalities=gpd.read_file(wfs_url)

**Bonus** : If you want to explore a GeoDataframes by displaying it on a map with leaflet/folium you can use the ```explore()``` function.

Note : Explore don't like the Timestamp type, so we exclude the corresponding columns.
```
municipalities.loc[:,~municipalities.columns.isin(['date_creation', 'date_modification'])].explore()
```

In [ ]:
municipalities.loc[:,~municipalities.columns.isin(['date_creation', 'date_modification'])].explore()

## Find the row corresponding to "La Teste-de-Buch" municipalites then use the plot() function to show the zone.

<details>

<summary>Tips</summary>

You can use the columns property of the geopandas to look for columns that could help you filter the rows.
```
print(municipalities.columns)
```
For non-native speaker, "nom_officiel" is the official name of the municipalities (a good starting point ?). "code_insee" is a unique code associated to each municipalities. Most of the time you can find this code on internet : https://www.google.com/search?q=la+teste+de+buch+code+insee
</details>

<details>

<summary>Help : I have 0 results... :(</summary>

Check that your test is valid. Is the municipalities name exactly the one expected ? Is the code_insee a number or a string ?
</details>

In [ ]:
print(municipalities.columns)

In [ ]:
#Your code here
area_of_interest= municipalities[municipalities['nom_officiel'] == 'La Teste-de-Buch']

#You can use
display(area_of_interest.iloc[0]["geometry"])
#or
area_of_interest.plot()

Check the CRS of your row (``display(area_of_interest.crs)``). You may need this information later 😊

In [ ]:
#Your code here
display(area_of_interest.crs)

## Extract the bounding box of the zone (we don't need to give a complex geometry to the search function)

<details>

<summary>Tips</summary>

**bounds** will give you the bounds of each row, you then need to select the first one

**total_bounds** will give you the bounds of all the rows (even if their is only one in our case 😛)
</details>

In [ ]:
import shapely

#Your code here
aoi_bbox = area_of_interest.total_bounds    # The bounding box of your area of interest. Which bounds will you choose ?

aoi_bbox_rect=[(aoi_bbox[1],aoi_bbox[0]), (aoi_bbox[3],aoi_bbox[2])] # [(miny,minx), (maxy,maxx)] : Lat,Lon means that y is first
aoi_bbox_polygon = shapely.geometry.box(*aoi_bbox)
display(aoi_bbox)
display(aoi_bbox_rect) # As two points Northeast and Southwest
print(aoi_bbox_polygon) # As a polygon
display(aoi_bbox_polygon)

## Bonus : Verifying what we are doing (with folium)

In [ ]:
import folium
folium_tile='OpenStreetMap' # Default
#folium_tile='stamenwatercolor' # For Artists
#folium_tile='CartoDB Positron' # Discrete
#folium_tile='CartoDB Dark_matter' # Paint it black

map_osm = folium.Map(tiles=folium_tile)
map_osm.add_child(folium.GeoJson(data=aoi_bbox_polygon, style_function=lambda x: {'color': 'blue'})) # GeoJson need the Polygon format
map_osm.add_child(folium.GeoJson(data=area_of_interest["geometry"].to_json(), style_function=lambda x: {'color': 'black','fillColor': 'orange'}))
map_osm.fit_bounds(aoi_bbox_rect) # fit bounds need the NE-SW format
map_osm

# 2. Searching relevant images in the Copernicus catalog

The objective for Copernicus program from ESA (European Space Agency) is to **use vast amount of global data from satellites** and from ground-based, airborne and seaborne measurement systems **to produce timely and quality information, services and knowledge**, and to provide autonomous and independent access to information in the domains of environment and security on a global level in order to help service providers, public authorities and other international organizations improve the quality of life for the citizens of Europe. In other words, it pulls together all the information obtained by the Copernicus environmental satellites, air and ground stations and sensors to provide a comprehensive picture of the "health" of Earth.

One of the benefits of the Copernicus programme is that the data and information produced in the framework of Copernicus are made available **free-of-charge** to all its users and the public, thus allowing downstream services to be developed.

Source : https://en.wikipedia.org/wiki/Copernicus_Programme

Note it exists several access to these Data but that **Copernicus Data Space Ecosystem is the new way to follow** (since Jan 2023) and that other channel will be deprecated at the End of Sept. 2023.

Sources : 

https://dataspace.copernicus.eu/news/2023-7-13-accessing-sentinel-mission-data-new-copernicus-data-space-ecosystem-apis

https://github.com/eu-cdse/notebook-samples/blob/c0e0ade601973c5d4e4bf66a13c0b76ebb099805/sentinelhub/migration_from_scihub_guide.ipynb


## OData
OData (Open Data Protocol) is a standard that specifies a variety of best practices for creating and using REST APIs. OData makes it possible to build REST-based data services that let Web clients publish and edit resources that are recognized by Uniform Resource Locators (URLs) and described in a data model using straightforward HTTP messages. This is the method you will want to use if your workflow requires the download of **full products**/granules/tiles. 

⚠ Full products archive are generally around 1Go each. **Downloading them require generaly a long time.**

The documentation regarding this can be found [here](https://documentation.dataspace.copernicus.eu/APIs/OData.html).

In this example, we will search the catalogue, generate the required credentials and then download a Sentinel-2 L2A granule using this protocol:

### Setting our search parameters

Firstly, we need to define our `start_date` and `end_date`, the `data_collection` and the area of interest (`aoi`). We define them in the next cell and will insert them into our request as string variables.

**For the moment, we will consider the period before the fire**, let say from start of April to end of June 2022.

In [ ]:
#Your code here
start_date = "2022-04-01"
end_date = "2022-06-30"
data_collection = "SENTINEL-2"
product_type = "S2MSI2A" # Level 2A products (cf. https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a)
print(aoi_bbox_polygon)  # Set from previous section

To search the catalogue we use the following code block:

In [ ]:
import pandas as pd 
import requests
url="https://catalogue.dataspace.copernicus.eu/odata/v1/Products?" \
+"$filter=Collection/Name eq '{}'".format(data_collection) \
+" and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{}')".format(product_type) \
+" and OData.CSC.Intersects(area=geography'SRID=4326;{}')".format(aoi_bbox_polygon) \
+" and ContentDate/Start gt {}T00:00:00.000Z".format(start_date) \
+" and ContentDate/Start lt {}T00:00:00.000Z".format(end_date) \
+"&$expand=Assets" \
+"&$expand=Attributes" \
+"&$orderby=ContentDate/Start" \
+"&$top=50"
print(url)
product_json = requests.get(url).json()
if "detail" in product_json: #Error int the requests
    print(product_json)
else:
    products = pd.DataFrame.from_dict(product_json['value'])
    print("Found: {}".format(len(products)))
    display(products)
    display(products.columns)

### Question : How many image are available for your criteria ? What is the date of the image acquisition ? The cloud coverage ?

The Cloud cover information is hidden in the Attributes field. Use the following line to extract it :

In [ ]:
products["cloudCover"] = [attribute["Value"] for attribute in products["Attributes"].explode() if attribute["Name"] == 'cloudCover'] # Dark magic. You may try to execute by pieces to understand how it works
products[["OriginDate","cloudCover"]]

Question : What can you say about the proportion of **usable** images ?  
I think it's not considered usable until the cloud coverage is less than twenty or thirty percent.

The catalog contains a Quicklook image. 

The url to the image is hidden in the Assets field.
```
products["Assets"].iloc[idx][0]['DownloadLink'] # Copernicus arbitray Assets field organisation
```

Once displayed, Ctrl+Click on it to download it with you web browser

In [ ]:
#Your code here
idx = 10
idx_url = products["Assets"].iloc[idx][0]['DownloadLink']
print(idx_url)

The following function can be used to download and display remote images:

In [ ]:
import requests
from PIL import Image
from io import BytesIO
def showRemoteImage(image_url):
    # Download the image using requests
    my_res = requests.get(image_url)
    if my_res.status_code == 200:
        # Open the downloaded image in PIL
        my_img = Image.open(BytesIO(my_res.content))
        # Show the image
        display(my_img)
    else:
        display(my_res)

Use it to display the Quicklook in your notebook.

In [ ]:
#Your code here
showRemoteImage(idx_url)

ipywidget is a library that you can use to have interactive user interface.

We can combine it with the previous function, to obtain a nice usable tool.

In [ ]:
import ipywidgets

def selectionWidget(products,selectedColumns):
    def loadQuickLook(idx=0):
        showRemoteImage(products["Assets"].iloc[idx][0]['DownloadLink']) 
        return products[selectedColumns].iloc[idx]
    ipywidgets.interact(loadQuickLook,  idx=(0, products.shape[0]-1))

selectionWidget(products,['OriginDate','cloudCover','Online'])

### Wouldn't it be more efficient to filter on cloudCover directly ?

You're right.
In a new cell, copypaste the code of the request and **add the following criteria to your url**.
```
and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10.00)
```
(i.e. we want a cloudCover value < 10.0)

In [ ]:
#Your code here
idx_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?" \
+ "$filter=Collection/Name eq '{}'".format(data_collection) \
+ " and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{}')".format(product_type) \
+ " and OData.CSC.Intersects(area=geography'SRID=4326;{}')".format(aoi_bbox_polygon) \
+ " and ContentDate/Start gt {}T00:00:00.000Z".format(start_date) \
+ " and ContentDate/Start lt {}T00:00:00.000Z".format(end_date) \
+ " and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10.00)" \
+ "&$expand=Assets" \
+ "&$expand=Attributes" \
+ "&$orderby=ContentDate/Start" \
+ "&$top=50"

product_json = requests.get(idx_url).json()
if "detail" in product_json: #Error int the requests
    print(product_json)
else:
    products = pd.DataFrame.from_dict(product_json['value'])
    print("Found: {}".format(len(products)))
    display(products)
    display(products.columns)

<details>

<summary>Tips</summary>
You cannot add the criteria anywhere in the request. Check how the <verb>$filter</verb> part is constructed.

Also, don't forget the \ at the end of the line (that allow to break the line in several lines for lisibility)
</details>

Extract the cloudCover column and show the Quicklook of the image in the ipywidget (Only the minimum necessary copypasting)

In [ ]:
#Your code here
# 导入所需的库
import ipywidgets

# 创建一个函数，用于显示产品的快速预览和提取 cloudCover 列的值
def displayQuicklookAndCloudCover(products):
    def loadQuickLook(idx=0):
        # 提取 cloudCover 值
        cloud_cover = products["cloudCover"].iloc[idx]
        # 如果 cloudCover 值小于等于 10.0，则显示快速预览图像
        if cloud_cover <= 10.0:
            showRemoteImage(products["Assets"].iloc[idx][0]['DownloadLink'])
        else:
            print("Cloud cover too high for quicklook display.")

        return cloud_cover

    # 创建交互式小部件，允许用户选择产品
    ipywidgets.interact(loadQuickLook, idx=(0, products.shape[0] - 1))

# 调用函数，传入产品数据帧
displayQuicklookAndCloudCover(products)


### A Little detour : productType

- Copypaste the request cell, but **remove the productType criteria**.
- Extract the cloudCover.
- Duplicate the process to also extract the productType from the Attribute field.
- Modify the call to selectionWidget to add the "productType" column in the information displayed
- Show the query result in the ipywidget

In [ ]:
#Your code here
url_without_product_type = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?" \
    + "$filter=Collection/Name eq '{}'".format(data_collection) \
    + " and OData.CSC.Intersects(area=geography'SRID=4326;{}')".format(aoi_bbox_polygon) \
    + " and ContentDate/Start gt {}T00:00:00.000Z".format(start_date) \
    + " and ContentDate/Start lt {}T00:00:00.000Z".format(end_date) \
    + " and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10.00)" \
    + "&$expand=Assets" \
    + "&$expand=Attributes" \
    + "&$orderby=ContentDate/Start" \
    + "&$top=50"

product_json_without_product_type = requests.get(url_without_product_type).json()
if "detail" in product_json_without_product_type: # 请求出错
    print(product_json_without_product_type)
else:
    products_without_product_type = pd.DataFrame.from_dict(product_json_without_product_type['value'])

# 提取云覆盖率
products_without_product_type['cloudCover'] = [attribute["Value"] for attribute in products_without_product_type["Attributes"].explode() if attribute["Name"] == 'cloudCover']

# 从 Attributes 字段中提取 productType
products_without_product_type['productType'] = products_without_product_type['Attributes'].apply(lambda x: next((item['Value'] for item in x if item['Name'] == 'productType'), None))

# 调用 selectionWidget 函数，同时显示 productType 列的信息
selectionWidget(products_without_product_type, ['OriginDate', 'cloudCover', 'productType'])


Questions :

- What is the main visible difference between the level 1C and level 2A ?
- If I was a bird, what would I see ?
- If I was a high altitude plane, what would I see ?
- What is the most helpful for my application ?

**Note the Id of the desired image and store it for later.**

In [ ]:
#Your code here
before_id = ...

# 3. Actually downloading the images from the Sentinel Dataspace

### The access to the catalog is public, but you need to be registred to download the data.

1. Follow the procedure here to create a free account on Copernicus Dataspace : https://documentation.dataspace.copernicus.eu/Registration.html (Registration + Email validation)
2. Create a ```myCopernicusCredentials.py``` file with only two lines :
```
username="name@email.com"
password="XXXXXXXXX"
```
**⚠ DON'T UPLOAD YOUR CREDENTIAL FILE ON MOODLE. IT IS PERSONNAL AND SECRET !**

3. Create an access token with your user/password with the following ```get_keycloak()``` function
4. Use the access token to download the image archive

**⚠ The archive is really heavy (~1.3Go). You don't need to go until the end of the download** (Square -> interrupt). Just show that you started the download.


In [ ]:
import requests
import os
from tqdm.notebook import tqdm
# Function inspired by Copernicus documentation : 
# - https://documentation.dataspace.copernicus.eu/APIs/Token.html
# - https://documentation.dataspace.copernicus.eu/APIs/OData.html#product-download

def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
        }
    try:
        r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
        data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
            )
    return r.json()["access_token"]       

def downloadCopernicusFile(product_id, keycloak_token):
    session = requests.Session()
    session.headers.update({'Authorization': f'Bearer {keycloak_token}'}) # Authorization header
    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    response = session.get(url, allow_redirects=False, stream=True)
    while response.status_code in (301, 302, 303, 307):
        url = response.headers['Location']
        response = session.get(url, allow_redirects=False, stream=True)

    file = session.get(url, verify=False, allow_redirects=True, stream=True)
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    if os.path.exists(f"{product_id}.zip"):
        print("ERROR, file already exists")
        return
    with open(f"{product_id}.zip", 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")

In [ ]:
import myCopernicusCredentials

#Your code here
keycloak_token = ... 
downloadCopernicusFile(...)

<details>

<summary>Tips</summary>
Once imported you can use ```myCopernicusCredentials.username``` and ```myCopernicusCredentials.password```
</details>

**⚠ Note: Previous API had the possibility to download only a part of the archive, but it is not possible currently this the ODATA API to the Copernicus Dataspace.**

You will find on Moodle a ```S2B_MSIL2A_20220528T105619_N0400_R094_T30TXQ_20220528T130057.SAFE_subset.zip``` archive where you will find a expurged archive with only the B02,B03,B04,B8A channel at 20m resolution (to keep it "small").

**If you achieved to download the whole archive, you don't need the Moodle file**, if not, it is a way to continue the Lab.

# 4. Image processing on Sentinel-2 images :  True-color, False-color

Using what you learn in the Image Processing Capsule:
1. Load the three 16-bits images with cv2
2. Apply a x3.5 factor on the brightness of the channels
3. Merge them into a RGB 16-bits image with `np.dstack()`
4. Display the image as a 8-bit image in the notebook

Do the process twice:
- once for a True-color result (B04 as red, B03 as green, B02 as blue).
- once for a False-color result (B8A as red, B04 as green, B03 as blue).

In [ ]:
#Your code here

# 5. High level processing with SentinelHub library

## Accessing data via Sentinel Hub APIs

The [Sentinel Hub API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub.html) is a RESTful API interface that provides access to various satellite imagery archives. It allows you to access raw satellite data, [rendered images](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html), [statistical analysis](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Statistical.html), and other features. 

In these examples, we will be using the `sentinelhub` python [package](https://sentinelhub-py.readthedocs.io/en/latest/index.html). The sentinelhub Python package is the official Python interface for [Sentinel Hub](https://www.sentinel-hub.com/) services. The package provides a collection of basic tools and utilities for working with geospatial and satellite data. It builds on top of well known packages such as `numpy`, `shapely`, `pyproj`.

To successfully run this notebook, make sure that you install or upgrade ```sentinelhub``` package to at least Version `3.9.1`.

### Credentials

**Nota Bene**: Even SentinelHub try to access the same Copernicus Dataspace, it will require an additional authentification step : the creation of a unique pair client_id/client_secret *dedicated to this API*.

To obtain your `client_id` & `client_secret` you need to navigate to your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). 
*(You may require to login with the username/password created in the first part of the Lab)*

In the User Settings you can create a new OAuth Client to generate these credentials. For more detailed instructions, visit the relevent [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Instructions on how to configure your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions you can create a profile specific to using the package for accessing Copernicus Data Space Ecosystem data collections.

Add `client_id` and `client_secret` to your `myCopernicusCredentials.py` file.

<details>

<summary>Help myCopernicusCredentials.client_id doest not exists !</summary>
1. Have you added those variable to the myCopernicusCredentials.py file ?
2. You may need to restart the Jupyter kernel if you have already imported it previoulsy (probably your case)
</details>

In [ ]:
import myCopernicusCredentials
import sentinelhub

config = sentinelhub.SHConfig()

config.sh_client_id = myCopernicusCredentials.client_id
config.sh_client_secret =  myCopernicusCredentials.client_secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")
# Saved config can be later accessed with config = SHConfig("cdse")

config = sentinelhub.SHConfig("cdse")

### Setting an area of interest

The bounding box in `WGS84` coordinate system is `[(longitude and latitude coordinates of lower left and upper right corners)]`. 

If we compare to the previous method where we needed `aoi_bbox_polygon`, here, we only need the simple `aoi_bbox` (created from the `bounds`/`total_bounds` attribute.)

**Note**: In the case of a general project, you can find the bounding box of anything zone with the help of the [bboxfinder](http://bboxfinder.com/) website.

All requests require a bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.constants.CRS`). In our case it is in WGS84 and we can use the predefined WGS84 coordinate reference system from `sentinelhub.constants.CRS`.

In [ ]:
#If the aoi_bbox value is still in memory
display(type(aoi_bbox),aoi_bbox)
#If not, just re-run the corresponding cells ("Find an Area of Interest" section).

When the bounding box bounds have been defined, you can initialize the `BBox` of the area of interest. Using the `bbox_to_dimensions` utility function, you can provide the desired resolution parameter of the image in meters and obtain the output image shape.

In [ ]:
resolution = 20
aoi_bbox_sh = sentinelhub.BBox(bbox=list(aoi_bbox), crs=sentinelhub.CRS.WGS84)
aoi_size = sentinelhub.bbox_to_dimensions(aoi_bbox_sh, resolution=resolution)

print(f'Image shape at {resolution} m resolution: {aoi_size} pixels')

### Catalog API
To search and discover data, you can use the Catalog API. Sentinel Hub Catalog API (or shortly "Catalog") is an API implementing the STAC Specification, providing geospatial information for data available in Sentinel Hub. Firstly, to initialise the `SentinelHubCatalog` class we will use:

In [ ]:
catalog = sentinelhub.SentinelHubCatalog(config=config)

Now we can build the Catalog API request; to do this we use the `aoi_bbox_sh` we defined earlier as well as `time_interval` and insert these into the request:

**Note**: `start_date` and `end_date` were set in the "Searching relevant images in the Copernicus catalog" section. You can rewrite them here for simplicity

In [ ]:
#start_date=...
#end_date=...
time_interval = start_date, end_date

search_iterator = catalog.search(
    sentinelhub.DataCollection.SENTINEL2_L2A, # Level 2A from Sentinel-2
    bbox=aoi_bbox_sh,                         # Bounding box of the Area of Interest
    time=time_interval,                       # Time interval
    filter="eo:cloud_cover < 10",             # Cloud cover < 10%
)

for row in search_iterator:
    print(row)

results = list(search_iterator)
print("Total number of results:", len(results))

results

We can see that even if we globally filter on the same parameters, the interface is much easier. And the actual cloud_cover value of the image is directly accessible.

**In the other hand, no Quicklook in this catalog (for the moment?), so our OData Catalog search is still relevant.**

### Where Sentinel Hub will really shine : the Processing API

Sentinel Hub have several API : Catalog, Statisical ... and Processing (cf. https://docs.sentinel-hub.com/api/latest/api/overview/).

The Processing API allows you to **send a "script" to be evaluated in the cloud on the specified data** then the function return you directly the result of the calculation.

Better, you don't have to specify a tile: if your bounding box is over several tiles, it will stich the corresponding tiles itself.

Even better, if you give him a time interval, it can choose the less cloudy pixels from the period to make the calculation. (*Note that it means that you may have a resulting image that is composed of pixels from differents date !*)

#### Example 1: True Color Image

We build the request according to the [API Reference](https://docs.sentinel-hub.com/api/latest/reference/), using the `SentinelHubRequest` class. Each Process API request also needs an [evalscript](https://docs.sentinel-hub.com/api/latest/#/Evalscript/V3/README).

The information that we specify in the `SentinelHubRequest` object is:
- an evalscript,
- a list of input data collections with time interval,
- a format of the response,
- a bounding box and it’s size (size or resolution).
- `mosaickingOrder` (optional): in this example we have used `leastCC` which will return pixels from the least cloudy acquisition in the specified time period.

The evalscript in the example is used to select the appropriate bands. We return the RGB (B04, B03, B02) Sentinel-2 L2A bands.

The least cloudy image from the time period is downloaded. Without any additional parameters in the evalscript, the downloaded data will correspond to reflectance values in `UINT8` format (values in 0-255 range).

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = sentinelhub.SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        sentinelhub.SentinelHubRequest.input_data(
            data_collection=sentinelhub.DataCollection.SENTINEL2_L2A.define_from(
                name="s2", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=(start_date, end_date),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}}           )
    ],
    responses=[sentinelhub.SentinelHubRequest.output_response("default", sentinelhub.MimeType.PNG)],
    bbox=aoi_bbox_sh,
    size=aoi_size, # WARNING, the output size should be < (2500,2500)
    config=config,
)

The method `get_data()` will always return a list of length 1 with the available image from the requested time interval in the form of numpy arrays.

In [ ]:
true_color_imgs = request_true_color.get_data()

print(f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}.")
print(f"Single element in the list is of type {type(true_color_imgs[-1])} / {true_color_imgs[-1].dtype} and has shape {true_color_imgs[-1].shape}")

Display the true_color_imgs (after applying a x3.5 factor on brightness for readability)

In [ ]:
#Your code here

# 6. Application to the analysis of the fire in La Teste-de-Buch 

Use the Sentinel Hub Processing API to produce:
- For the time interval BEFORE the fire and AFTER the fire :
  - The true color and false color images
  - The NDVI ratio (Nice example at the end of this [Notebook](https://github.com/eu-cdse/notebook-samples/blob/c0e0ade601973c5d4e4bf66a13c0b76ebb099805/sentinelhub/migration_from_scihub_guide.ipynb), another proposition in this great [Custom scripts collections](https://custom-scripts.sentinel-hub.com/custom-scripts/hls/ndvi/))
  - The Normalized Burn Ratio.  

The formula of this last one is : **(NIR - SWIR) / (NIR + SWIR)**  

If we want to use B12 with B08, we need to upsample the array. We could also use B8A which is 20m, but we want maximum precision.  

If you want, you can combine further the resulting images.

You can also make some statistical calculation (% of the area burned per ex.)

**Then, write a report based on what you can see/say on this situation**

# Second part of the lab : Do your own analysis !

Find a subject (deforestation, oil spil, algae blooming, fire in canada/portugal/etc.) and make a study based on SENTINEL-2 images.

Explain your idea and how you develop the pipeline adequately. Daring will be rewarded. (Choose fire detection in other region only if you don't feel confident on other subjects.)